End-to-End Video Processing 

My goal for this notebook is to use the new `daft.File` type. 

In this use caes we want both the image embeddings as well as the audio. 


## Setup

Setting this project up is a pain in the butt. Theres issues with the original repo with tennsorflow-cpu. just delete the "-cpu" and most stuff works. I did have to clone the repo inside this one though. 

### Demo Notebook's Utilities vs daft is ridiculous

In [1]:
# @title Load dependencies and define utilities

import mediapy
import numpy as np

def read_and_preprocess_video(
    filename: str, target_num_frames: int, target_frame_size: tuple[int, int]
):
  """Reads and preprocesses a video."""

  frames = mediapy.read_video(filename)

  # Sample to target number of frames.
  frame_indices = np.linspace(
      0, len(frames), num=target_num_frames, endpoint=False, dtype=np.int32
  )
  frames = np.array([frames[i] for i in frame_indices])

  # Resize to target size.
  original_height, original_width = frames.shape[-3:-1]
  target_height, target_width = target_frame_size
  assert (
      original_height * target_width == original_width * target_height
  ), 'Currently does not support aspect ratio mismatch.'
  frames = mediapy.resize_video(frames, shape=target_frame_size)

  # Normalize pixel values to [0.0, 1.0].
  frames = mediapy.to_float01(frames)

  return frames

In [2]:
import daft
from daft import col

# Read and preprocess video
def read_and_preprocess_video_daft(
    filenames: list[str], target_num_frames: int, target_frame_size: tuple[int, int]
):      
    df = daft.read_video_frames(
        filenames, 
        image_height=target_frame_size[0], 
        image_width=target_frame_size[1]
    ) 
    df = (
        df
        .with_column("frame_group_index", col("frame_index") // target_num_frames)
        .groupby("frame_group_index").agg_list("data")
    )
    return df

In [3]:
filenames = ['/Users/everett-founder/git/dream/daft-video-embeddings/videoprism/videoprism/assets/water_bottle_drumming.mp4']
target_num_frames = 16
target_frame_size = (288, 288)

#frames = read_and_preprocess_video(filenames[0], target_num_frames, target_frame_size)
df_frames = read_and_preprocess_video_daft(filenames, target_num_frames, target_frame_size)

#print(frames)
df_frames.show()

🗡️ 🐟 Project: 00:00 

🗡️ 🐟 GroupedAggregate: 00:00 

🗡️ 🐟 Limit 8: 00:00 

🗡️ 🐟 PythonFunction Scan: 00:00 

Error when running pipeline node PythonFunction Scan


DaftCoreException: DaftError::External PyIOError: EOFError: [Errno 541478725] End of file: 'avcodec_send_packet()'

Um ok. what? Lets try this again as a script.... ok so this just doesn't want to load these images? I guess I don't need to group by... I can just query the frames I need when I need them. 

Oh shoot I also need to normalize the pixels... and cast to int32

 ok lets break this down step by step.

In [1]:
# read the frames (also resize, thanks connor)
df = daft.read_video_frames(
        filenames, 
        image_height=target_frame_size[0], 
        image_width=target_frame_size[1]
)
df.show(3) 


NameError: name 'daft' is not defined

In [16]:
# Assign frame groups
df = df.with_column("frame_group_index", col("frame_index") // target_num_frames)
df.select("path", "frame_group_index", "frame_index", "data").show()

pathUtf8,frame_group_indexInt64,frame_indexInt64,dataImage[RGB; 288 x 288]
file:///Users/everett-founder/git/dream/daft-video-embeddings/videoprism/videoprism/assets/water_bottle_drumming.mp4,0,0,""" />"
file:///Users/everett-founder/git/dream/daft-video-embeddings/videoprism/videoprism/assets/water_bottle_drumming.mp4,0,1,""" />"
file:///Users/everett-founder/git/dream/daft-video-embeddings/videoprism/videoprism/assets/water_bottle_drumming.mp4,0,2,""" />"
file:///Users/everett-founder/git/dream/daft-video-embeddings/videoprism/videoprism/assets/water_bottle_drumming.mp4,0,3,""" />"
file:///Users/everett-founder/git/dream/daft-video-embeddings/videoprism/videoprism/assets/water_bottle_drumming.mp4,0,4,""" />"
file:///Users/everett-founder/git/dream/daft-video-embeddings/videoprism/videoprism/assets/water_bottle_drumming.mp4,0,5,""" />"
file:///Users/everett-founder/git/dream/daft-video-embeddings/videoprism/videoprism/assets/water_bottle_drumming.mp4,0,6,""" />"
file:///Users/everett-founder/git/dream/daft-video-embeddings/videoprism/videoprism/assets/water_bottle_drumming.mp4,0,7,""" />"


In [ ]:
frames[0]

array([0.6901961, 0.7019608, 0.654902 ], dtype=float32)

In [20]:
frames[0][0]

array([[0.6901961 , 0.7019608 , 0.654902  ],
       [0.26666668, 0.27058825, 0.22745098],
       [0.25882354, 0.23921569, 0.20784314],
       [0.29803923, 0.27058825, 0.23137255],
       [0.3019608 , 0.27450982, 0.23529412],
       [0.28627452, 0.25490198, 0.21568628],
       [0.28627452, 0.2627451 , 0.21176471],
       [0.29803923, 0.27058825, 0.22352941],
       [0.30588236, 0.27058825, 0.22745098],
       [0.31764707, 0.27058825, 0.23137255],
       [0.32156864, 0.27450982, 0.23529412],
       [0.3372549 , 0.2901961 , 0.2509804 ],
       [0.34901962, 0.3019608 , 0.2627451 ],
       [0.3647059 , 0.31764707, 0.2784314 ],
       [0.37254903, 0.3254902 , 0.28627452],
       [0.41960785, 0.37254903, 0.33333334],
       [0.49411765, 0.44705883, 0.40784314],
       [0.7294118 , 0.68235296, 0.6431373 ],
       [0.76862746, 0.73333335, 0.6901961 ],
       [0.6509804 , 0.61960787, 0.57254905],
       [0.6156863 , 0.5921569 , 0.5411765 ],
       [0.627451  , 0.6       , 0.5529412 ],
       [0.

In [21]:
frames[0][0][0]

array([0.6901961, 0.7019608, 0.654902 ], dtype=float32)

uh huh... so RGB?

In [29]:
df = df.with_column("image_pixels", col("data").image.to_mode("RGB").cast(daft.DataType.int32))

df.select("image_pixels").show()

AssertionError: 

Nope. If I was a native image type. How would I want to be handled... Fine I'll look at the docs. 

Ok so I'm looking at the misc docs for the image modes and image formats and I'm assuming that BMP means BitMap... but I can't tell because there is no description. 

Oh so there are 8 bit modes? No idea what L, L16, LA, LA16, or the other things are. I just know I need my pixels to be from 0 to 1. 

ASK the superintelligence! 

ME: Hey so I am trying to figure out if this image preprocessing that google is doing for videoprism (taking the rgbs and converting them into floats from 0 to 1), if that has a name or is a specific format type. Daft has these modes and formats that they support for images but I don't know what they mean. 

Heres the docs 

<misc docs>

GPT: 
I'll look for how VideoPrism converts frames to floats and whether it's described as a specific mode (e.g., RGB32F) in the code, then map that to Daft's image modes.

[1 tool called]

I'll scan the local VideoPrism code for where frames are converted to float and normalized to [0,1], then map that to Daft's image modes.

[5 tools called]

I checked VideoPrism’s docs/code: it expects RGB pixel values normalized to [0.0, 1.0] (float32), not a special color space.

```104:108:videoprism/README.md
The RGB values of input videos should be normalized in [0.0, 1.0].
```

```119:123:videoprism/videoprism/colabs/videoprism_video_text_demo.ipynb
# Normalize pixel values to [0.0, 1.0].
frames = mediapy.to_float01(frames)
```

### What this is called
- It’s just per‑channel normalization to [0,1] in float32.
- Common names: “float32 RGB in [0,1]”, “normalized to [0,1]”, sometimes “ToTensor-style normalization”.
- It does not by itself change color space or gamma; it remains sRGB unless otherwise specified.

### Mapping to Daft
- Daft “modes” describe channel layout and bit depth:
  - **RGB**: 8‑bit per channel (0–255)
  - **RGB16**: 16‑bit integer per channel
  - **RGB32F**: 32‑bit float per channel (values typically in [0,1])
  - Likewise for L/LA/RGBA variants
- Daft “formats” are file containers: **PNG**, **JPEG**, **TIFF**, etc.

So Google’s preprocessing corresponds to Daft’s mode: **ImageMode.RGB32F** (or **RGBA32F** if alpha is present). The value range [0,1] matches typical RGB32F usage.

Note: Daft currently only stores 8‑bit modes (`L`, `LA`, `RGB`, `RGBA`) in a DataFrame. If you need float32 normalized tensors:
- Decode/store as `RGB`, then convert to float32 in your computation: divide by 255.0.
- Or decode to `RGB32F` at read time if supported, but you’ll keep it in memory for processing rather than storing it directly in a DataFrame column.

If you share your exact Daft read/transform snippet, I can suggest the minimal change to mirror VideoPrism’s expected [0,1] float32 normalization.



In [34]:
# ok so I need to decode to RGB and then convert to float32 in my computation. 
# its already rgb... and casting to int32 doesn't work... what if I just query the pixels in the udf for inference?

df = df.with_column("data", col("data").image.to_mode("RGB32F"))

df_1 = df.where(col("frame_index") == 0).select("data").limit(1)
df_1.collect()


/Users/everett-founder/git/dream/daft-video-embeddings/.venv/lib/python3.11/site-packages/daft/dashboard/__init__.py:91: UserWarning: Failed to broadcast metrics over http://127.0.0.1:3238/api/queries: HTTP Error 400: Bad Request
  warnings.warn(f"Failed to broadcast metrics over {url}: {e}")


🗡️ 🐟 Limit 1: 00:00 

🗡️ 🐟 PythonFunction Scan: 00:00 

🗡️ 🐟 Project: 00:00 


thread 'DAFTCPU-6' (10365526) panicked at src/daft-core/src/array/ops/image.rs:174:5:
assertion failed: is_all_u8
Error when running pipeline node Project


DaftCoreException: DaftError::External task 293 panicked with message "assertion failed: is_all_u8"

In [9]:
# BRO... WHY - ok lets back up and try again. (restarted kernel just in case)
import daft
from daft import col
filenames = ['/Users/everett-founder/git/dream/daft-video-embeddings/videoprism/videoprism/assets/water_bottle_drumming.mp4']
target_num_frames = 16
target_frame_size = (288, 288)

df = daft.read_video_frames(
        filenames, 
        image_height=target_frame_size[0], 
        image_width=target_frame_size[1]
)
df = (
        df
        .with_column("data", col("data").image.to_mode("RGB"))
        .with_column("group_index", col("frame_index") // target_num_frames)
)

df_new = df.groupby("group_index").agg_list("frame_index")


In [10]:
df.collect()

🗡️ 🐟 Project: 00:00 

Error when running pipeline node PythonFunction Scan


DaftCoreException: DaftError::External PyIOError: EOFError: [Errno 541478725] End of file: 'avcodec_send_packet()'

I DO NOT uNDERSTANd!
this does not compute... 

if I look at the query images example...

We decode the image from a path and then run it through this udf... 

```python
import numpy as np
import PIL.Image
from PIL import ImageFilter


def magic_red_detector(img: np.ndarray) -> PIL.Image.Image:
    """Gets a new image which is a mask covering all 'red' areas in the image."""
    img = PIL.Image.fromarray(img)
    lower = np.array([245, 100, 100])
    upper = np.array([10, 255, 255])
    lower_hue, upper_hue = lower[0, np.newaxis, np.newaxis], upper[0, np.newaxis, np.newaxis]
    lower_saturation_intensity, upper_saturation_intensity = (
        lower[1:, np.newaxis, np.newaxis],
        upper[1:, np.newaxis, np.newaxis],
    )
    hsv = img.convert("HSV")
    hsv = np.asarray(hsv).T
    mask = np.all(
        (hsv[1:, ...] >= lower_saturation_intensity) & (hsv[1:, ...] <= upper_saturation_intensity), axis=0
    ) & ((hsv[0, ...] >= lower_hue) | (hsv[0, ...] <= upper_hue))
    img = PIL.Image.fromarray(mask.T)
    img = img.filter(ImageFilter.ModeFilter(size=5))
    return img


df = df.with_column(
    "red_mask",
    df["image"].apply(magic_red_detector, return_dtype=daft.DataType.python()),
)

df.collect()

```
Does this work? 


In [13]:
import numpy as np
import PIL.Image
from PIL import ImageFilter


def magic_red_detector(img: np.ndarray) -> PIL.Image.Image:
    """Gets a new image which is a mask covering all 'red' areas in the image."""
    img = PIL.Image.fromarray(img)
    lower = np.array([245, 100, 100])
    upper = np.array([10, 255, 255])
    lower_hue, upper_hue = lower[0, np.newaxis, np.newaxis], upper[0, np.newaxis, np.newaxis]
    lower_saturation_intensity, upper_saturation_intensity = (
        lower[1:, np.newaxis, np.newaxis],
        upper[1:, np.newaxis, np.newaxis],
    )
    hsv = img.convert("HSV")
    hsv = np.asarray(hsv).T
    mask = np.all(
        (hsv[1:, ...] >= lower_saturation_intensity) & (hsv[1:, ...] <= upper_saturation_intensity), axis=0
    ) & ((hsv[0, ...] >= lower_hue) | (hsv[0, ...] <= upper_hue))
    img = PIL.Image.fromarray(mask.T)
    img = img.filter(ImageFilter.ModeFilter(size=5))
    return img


In [14]:
df = df.with_column(
    "red_mask",
    df["data"].apply(magic_red_detector, return_dtype=daft.DataType.python()),
)

df.collect()

🗡️ 🐟 UDF magic_red_detector: 00:00 

🗡️ 🐟 Project: 00:00 

🗡️ 🐟 PythonFunction Scan: 00:00 

Error when running pipeline node PythonFunction Scan


DaftCoreException: DaftError::External PyIOError: EOFError: [Errno 541478725] End of file: 'avcodec_send_packet()'

uh oh. This isn't good. um... ok breaking this up... ugh... wait... 

In [15]:
df = daft.read_video_frames(
        filenames, 
        image_height=target_frame_size[0], 
        image_width=target_frame_size[1]
)

In [16]:
df_group_id = df.with_column("group_index", col("frame_index") // target_num_frames)
df_group_id.show()




pathUtf8,frame_indexInt64,frame_timeFloat64,frame_time_baseUtf8,frame_ptsInt64,frame_dtsInt64,frame_durationInt64,is_key_frameBoolean,dataImage[RGB; 288 x 288],group_indexInt64
file:///Users/everett-founder/git/dream/daft-video-embeddings/videoprism/videoprism/assets/water_bottle_drumming.mp4,0,0,1/12288,0,0,1024,true,""" />",0
file:///Users/everett-founder/git/dream/daft-video-embeddings/videoprism/videoprism/assets/water_bottle_drumming.mp4,1,0.08333333333333333,1/12288,1024,1024,1024,false,""" />",0
file:///Users/everett-founder/git/dream/daft-video-embeddings/videoprism/videoprism/assets/water_bottle_drumming.mp4,2,0.16666666666666666,1/12288,2048,2048,1024,false,""" />",0
file:///Users/everett-founder/git/dream/daft-video-embeddings/videoprism/videoprism/assets/water_bottle_drumming.mp4,3,0.25,1/12288,3072,3072,1024,false,""" />",0
file:///Users/everett-founder/git/dream/daft-video-embeddings/videoprism/videoprism/assets/water_bottle_drumming.mp4,4,0.3333333333333333,1/12288,4096,4096,1024,false,""" />",0
file:///Users/everett-founder/git/dream/daft-video-embeddings/videoprism/videoprism/assets/water_bottle_drumming.mp4,5,0.4166666666666667,1/12288,5120,5120,1024,false,""" />",0
file:///Users/everett-founder/git/dream/daft-video-embeddings/videoprism/videoprism/assets/water_bottle_drumming.mp4,6,0.5,1/12288,6144,6144,1024,false,""" />",0
file:///Users/everett-founder/git/dream/daft-video-embeddings/videoprism/videoprism/assets/water_bottle_drumming.mp4,7,0.5833333333333334,1/12288,7168,7168,1024,false,""" />",0


In [20]:
df_grouped = df_group_id.select("group_index", "frame_index")
df_grouped.show()

group_indexInt64,frame_indexInt64
0,0
0,1
0,2
0,3
0,4
0,5
0,6
0,7


In [22]:
df_grouped = df_grouped.groupby("group_index").agg_list("frame_index").collect()

/Users/everett-founder/git/dream/daft-video-embeddings/.venv/lib/python3.11/site-packages/daft/dashboard/__init__.py:91: UserWarning: Failed to broadcast metrics over http://127.0.0.1:3238/api/queries: HTTP Error 400: Bad Request
  warnings.warn(f"Failed to broadcast metrics over {url}: {e}")


🗡️ 🐟 Project: 00:00 

🗡️ 🐟 GroupedAggregate: 00:00 

Error when running pipeline node PythonFunction Scan


DaftCoreException: DaftError::External PyIOError: EOFError: [Errno 541478725] End of file: 'avcodec_send_packet()'

NOW THAT Doesn't make ANY Sense. This has to be a dependency issue. Which means the advice of ("make sure to pip install av" is potentially screwing this up) I'm going  to restart again and try a new kernel. 

OK TRY 3

In [3]:
import daft
from daft import col
filenames = ['/Users/everett-founder/git/dream/daft-video-embeddings/videoprism/videoprism/assets/water_bottle_drumming.mp4']
target_num_frames = 16
target_frame_size = (288, 288)

df = daft.read_video_frames(
        "/Users/everett-founder/git/dream/daft-video-embeddings/videoprism/videoprism/assets/water_bottle_drumming.mp4", 
        image_height=288, 
        image_width=288
)
df_prepped = (
        df
        .with_column("data", col("data").image.to_mode("RGB32F"))
        .with_column("group_index", col("frame_index") // target_num_frames)
)

In [10]:
!uv pip install av

Using Python 3.11.13 environment at: /Users/everett-founder/git/dream/daft-video-embeddings/.venv
Audited 1 package in 32ms


In [4]:
df_prepped.collect()


thread 'DAFTCPU-3' (11891650) panicked at src/daft-core/src/array/ops/image.rs:174:5:
assertion failed: is_all_u8
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


🗡️ 🐟 PythonFunction Scan: 00:00 

🗡️ 🐟 Project: 00:00 


thread 'DAFTCPU-6' (11891653) panicked at src/daft-core/src/array/ops/image.rs:174:5:
assertion failed: is_all_u8
Error when running pipeline node Project


DaftCoreException: DaftError::External task 30 panicked with message "assertion failed: is_all_u8"

# IT WAS THE VIDEO?!

Wait can we not throw images into a list? 

Let me think about this for a second. With a batch udf, we would grab the batch of images and perform inference, passing back the embedding results for that batch... but the input data structure is inherently batched. A row-wise udf following a group by makes a little more sense, considering the return type... 

This is the fundamental conflict between the tabular approach and processing videos. Are we just going to end up doing everything in a UDF? That doesnt sound very daft-native...

This is where we have to make a seperate dataframe right? using a different dataframe? 

In [2]:

def compute_similarity_matrix(
    video_embeddings,
    text_embeddings,
    temperature: float,
    apply_softmax: str | None = None,
) -> np.ndarray:
  """Computes cosine similarity matrix."""
  assert apply_softmax in [None, 'over_texts', 'over_videos']
  emb_dim = video_embeddings[0].shape[-1]
  assert emb_dim == text_embeddings[0].shape[-1]

  video_embeddings = np.array(video_embeddings).reshape(-1, emb_dim)
  text_embeddings = np.array(text_embeddings).reshape(-1, emb_dim)
  similarity_matrix = np.dot(video_embeddings, text_embeddings.T)

  if temperature is not None:
    similarity_matrix /= temperature

  if apply_softmax == 'over_videos':
    similarity_matrix = np.exp(similarity_matrix)
    similarity_matrix = similarity_matrix / np.sum(
        similarity_matrix, axis=0, keepdims=True
    )
  elif apply_softmax == 'over_texts':
    similarity_matrix = np.exp(similarity_matrix)
    similarity_matrix = similarity_matrix / np.sum(
        similarity_matrix, axis=1, keepdims=True
    )

  return similarity_matrix

NameError: name 'np' is not defined

In [ ]:
# @title Load model

import jax
import jax.numpy as jnp
from videoprism import models as vp

MODEL_NAME = 'videoprism_lvt_public_v1_base'  # @param ['videoprism_lvt_public_v1_base', 'videoprism_lvt_public_v1_large'] {allow-input: false}
NUM_FRAMES = 16
FRAME_SIZE = 288

flax_model = vp.get_model(MODEL_NAME)
loaded_state = vp.load_pretrained_weights(MODEL_NAME)
text_tokenizer = vp.load_text_tokenizer('c4_en')


@jax.jit
def forward_fn(inputs, text_token_ids, text_paddings, train=False):
  return flax_model.apply(
      loaded_state,
      inputs,
      text_token_ids,
      text_paddings,
      train=train,
  )

Already we run into confusion between audio and video. Given the io method of read_video_frames we could just read the video frames from that and write a seperate audio extraction file. 

For a use case where we are just looking to grab images, the read_video_frames makes sense, but if the proposal for the video type is going to be based on `File` then this feels funny. 

Oh... read_video_frames requires I specify image_height and width... I'm not sure what my video's height and width are natively. 

Well, my embedding models expect: 

In [ ]:
df = df.with_column("video_frames", daft.read_video_frames(df["file"], image_height=224, image_width=224))
df.limit(1).show()

TypeError: read_video_frames() missing 2 required positional arguments: 'image_height' and 'image_width'

# Hmm read video

In [ ]:
from daft import DataType as dt

@daft.func()
def ffmpeg(file: daft.File) -> dt.struct({""})